In [1]:
import torch
import torchvision
from torchvision import transforms
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [3]:
# 3*3 convolutino
def conv3x3(in_channels, out_channels, stride=1):
    return torch.nn.Conv2d(in_channels, out_channels, kernel_size=3,
                    stride=stride, padding=1, bias=False)


# Residual block
class ResidualBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = torch.nn.BatchNorm2d(out_channels)
        self.relu = torch.nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = torch.nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


# ResNet
class ResNet(torch.nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(1, 16)
        self.bn = torch.nn.BatchNorm2d(16)
        self.relu = torch.nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 1)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.layer4 = self.make_layer(block, 128, layers[1], 1)
        self.max_pool = torch.nn.MaxPool2d(8)
        self.fc = torch.nn.Linear(128, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = torch.nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                torch.nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return torch.nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.max_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [4]:
net = torch.load('FashionMNIST_ResNet.pth', map_location=torch.device('cpu'))
net = net.to('cuda:0')
net.eval()

ResNet(
  (conv): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

In [5]:
test_tf = transforms.Compose([
    transforms.ToTensor()
])

test = torchvision.datasets.FashionMNIST('', download=True, train=False, transform=test_tf)
testloader = torch.utils.data.DataLoader(dataset=test, batch_size=256, shuffle=False)

In [6]:
labels = {
0 : 'T-shirt/top',
1 : 'Trouser',
2 : 'Pullover',
3 : 'Dress',
4 : 'Coat',
5 : 'Sandal',
6 : 'Shirt',
7 : 'Sneaker',
8 : 'Bag',
9 : 'Ankle boot'
}

In [7]:
from numpy.linalg import norm
from scipy.special import softmax

examples = enumerate(testloader)
batch_idx, (example_data, example_targets) = next(examples)

In [8]:
train = torchvision.datasets.FashionMNIST('..', download=True, train=True, transform=test_tf)
trainloader = torch.utils.data.DataLoader(dataset=test, batch_size=10000, shuffle=False)

In [9]:
train_examples = enumerate(trainloader)
batch_idx, (train_data, train_targets) = next(train_examples)

In [10]:
from boundary_momentum import boundary_attack

In [11]:
def nearest_image(image, label, train_data, train_targets):
    image = image.data.cpu().numpy()[0]
    all_norm = []
    for i, train_image in enumerate(train_data):
        if train_targets[i].data.cpu().numpy() != label:
            train_image = train_image.data.cpu().numpy()
            diff = train_image - image
            norm_diff = norm(diff)
            all_norm.append(norm_diff)
        else:
            all_norm.append(1e20)            

    index = np.argmin(all_norm)
    return index

In [12]:
!nvidia-smi

Sat May  2 23:07:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:05:00.0 Off |                    0 |
| N/A   45C    P0    83W / 250W |   4118MiB / 16280MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

#### Boundary attack

In [ ]:
%%time

i = 0
k = 0

l1_norm = []
l2_norm = []
probs_init = []
probs_adv = []

while True:
    print('Attack:', i)
    class_label = example_targets[i].data.numpy()
    image = example_data[i]
    label = example_targets[i].data.numpy()

    prediction = net(torch.tensor(image.reshape(1, 1, 28, 28)).to('cuda:0')).to('cpu').data.numpy()
    prob_init = np.max(softmax(prediction))
    label_init = np.argmax(softmax(prediction))
    
    if label_init == label:
        index = nearest_image(image, label, train_data, train_targets)
        x_init = train_data[index].data.numpy().reshape(28,28)
        x_target = example_data[i].data.numpy().reshape(28,28)

        adv_example, proba = boundary_attack(net, x_init, x_target, threshold=None, verbose=1, max_iter=2e4, mu=0.1)

        l1 = np.linalg.norm(adv_example - x_target, ord=1)
        l2 = np.linalg.norm(adv_example - x_target, ord=2)
        if l2 > 1e-9:
            probs_init.append(prob_init)
            probs_adv.append(proba)
            l1_norm.append(l1)
            l2_norm.append(l2)
            np.save('probs_adv_1e-1.npy', probs_adv)
            np.save('l2_1e-1.npy', l2)
            np.save('l1_1e-1.npy', l1)
            k += 1

    i += 1
    
    if k >= 100:
        break

Attack: 0
Start boundary attack
distance: 0.1249, itetarion: 1874

In [ ]:
!nvidia-smi

In [ ]:
print(l1_norm)
print(l2_norm)
print(probs_init)
print(probs_adv)